# Day 11 - Monkey in the Middle

## Part 2
This is just some modifications, so doing as a seperate notebook to minimize amount of stuff I need to change.

In [25]:
from typing import Callable
from math import floor

class Troop:
    def __init__(self):
        self.members:[Monkey] = []    

    def throw(self, item:int, toMonkey:int):
        #manage stress
        item = item % self.commonModulo
        self.members[toMonkey].items.append(item)

    def monkeyBusiness(self)->int:
        #multiple together top two monkey's itemsInspected
        itemsInspected = [x.itemsInspected for x in self.members]
        itemsInspected.sort()
        return itemsInspected[0] * itemsInspected[1]

    def runRound(self):
        #run a single round
        for m in self.members:
            m.throwItems()
        

    def runXrounds(self, x:int):
        #another way to manage stress levels
        self.commonModulo = 1
        for m in self.members:
            self.commonModulo = self.commonModulo * m.testDiv

        
        for i in range(x):
            self.runRound()
            #if i%1000==0:
            if i%100==0:
                print('--Round: ' + str(i) + ' --')
                for m in range(len(self.members)):
                    print('Monkey ' + str(m) + ': ' + str(self.members[m].items))
        #calculate MonkeyBusiness: multiple two largest itemsInspected
        monkeyItems = []
        print('')
        for m in range(len(self.members)):
            print('Monkey '+str(m)+' inspected items '+str(self.members[m].itemsInspected)+' times')
            monkeyItems.append(self.members[m].itemsInspected)
        monkeyItems.sort(reverse=True)
        print(monkeyItems)
        monkeyBusiness = monkeyItems[0] * monkeyItems[1]
        print('Money Business: '+str(monkeyBusiness))

        


class Monkey:
    def __init__(self, troop:Troop, items:[int], op:Callable, testDiv:int, trueMonkey:int, falseMonkey:int):
        #passing the op as a function seems sensible... but might have to learn a bit more python to make that work!
        self.troop = troop
        self.items = items
        self.op = op
        self.testDiv = testDiv
        self.trueMonkey = trueMonkey
        self.falseMonkey = falseMonkey
        self.itemsInspected = 0

    def throwItems(self):
        #process each item
        for i in self.items:
            self.itemsInspected += 1
            #Operation to modify worry level
            i = self.op(i)
            #Divide worry level by 3, round down
            #comment out for part2 
            #i = floor(i/3)
            #Test & throw
            if i % self.testDiv == 0:
                self.troop.throw(i, self.trueMonkey)
            else:
                self.troop.throw(i, self.falseMonkey)

        #clear items
        self.items.clear()


In [26]:
# tests

tt = Troop()
tt.members.append(Monkey(troop= tt, 
  items= [79,98], 
  op= lambda x : x * 19, 
  testDiv= 23, 
  trueMonkey= 2, 
  falseMonkey= 3)
)
tt.members.append(Monkey(troop= tt, 
  items= [54, 65, 75, 74], 
  op= lambda x : x + 6, 
  testDiv= 19, 
  trueMonkey= 2, 
  falseMonkey= 0)
)
tt.members.append(Monkey(troop= tt, 
  items= [79, 60, 97], 
  op= lambda x : x * x, 
  testDiv= 13, 
  trueMonkey= 1, 
  falseMonkey= 3)
)
tt.members.append(Monkey(troop= tt, 
  items= [74], 
  op= lambda x : x + 3, 
  testDiv= 17, 
  trueMonkey= 0, 
  falseMonkey= 1)
)

tt.runXrounds(10000)
#tt.runXrounds(660)


--Round: 0 --
Monkey 0: [60, 71, 81, 80]
Monkey 1: [77, 1504, 1865, 6244, 3603, 9412]
Monkey 2: []
Monkey 3: []
--Round: 100 --
Monkey 0: [94382, 32138, 91722, 78042]
Monkey 1: [9522, 20504, 17179, 16134, 50942, 21188]
Monkey 2: []
Monkey 3: []
--Round: 200 --
Monkey 0: [74451, 4683, 32138, 49485, 68637, 72285, 72988]
Monkey 1: [63558, 66047, 55749]
Monkey 2: []
Monkey 3: []
--Round: 300 --
Monkey 0: [47794, 94553, 90772, 72988, 13379]
Monkey 1: [79081, 53773, 73552, 64185, 62342]
Monkey 2: []
Monkey 3: []
--Round: 400 --
Monkey 0: [69435, 13379]
Monkey 1: [68954, 20827, 77238, 64337, 11935, 90424, 95193, 53621]
Monkey 2: []
Monkey 3: []
--Round: 500 --
Monkey 0: [65407, 94667, 54425, 83932, 58054, 21004]
Monkey 1: [20827, 78416, 86738, 28275]
Monkey 2: []
Monkey 3: []
--Round: 600 --
Monkey 0: [58035, 5329, 83932, 22847, 29269, 28775]
Monkey 1: [858, 4905, 88049, 13094]
Monkey 2: []
Monkey 3: []
--Round: 700 --
Monkey 0: [60372, 49257, 91399, 22847, 79961]
Monkey 1: [74445, 16134, 475

# Performance issues!
Wasn't expecting this from such a modest change.
After around 600 rounds, the execution time is blowing up. Why? I assumed the execution time for round should be linear.

Why? Because the "worry level" of each time is blowing up. The clue is in the puzzel:
"Unfortunately, that relief was all that was keeping your worry levels from reaching ridiculous levels. You'll need to find another way to keep your worry levels manageable."

So what is this other way? 
Firstly, how is worry level used?
    It's used for the test division, to work out where to throw the item.

So my guess is this is a challenge about factorisation. 
Ooooh... we could multiple all the testDiv's together, and then modulo that.
Let's go back and try that.

Yes! It worked.


In [27]:
#Part 1 Real run
#And yes... I am going to cheat and skip the parsing of the code

rt = Troop()
rt.members.append(Monkey(troop= rt, 
  items= [78, 53, 89, 51, 52, 59, 58, 85], 
  op= lambda x : x * 3, 
  testDiv= 5, 
  trueMonkey= 2, 
  falseMonkey= 7)
)
rt.members.append(Monkey(troop= rt, 
  items= [64], 
  op= lambda x : x + 7, 
  testDiv= 2, 
  trueMonkey= 3, 
  falseMonkey= 6)
)
rt.members.append(Monkey(troop= rt, 
  items= [71, 93, 65, 82], 
  op= lambda x : x + 5, 
  testDiv= 13, 
  trueMonkey= 5, 
  falseMonkey= 4)
)
rt.members.append(Monkey(troop= rt, 
  items= [67, 73, 95, 75, 56, 74], 
  op= lambda x : x + 8, 
  testDiv= 19, 
  trueMonkey= 6, 
  falseMonkey= 0)
)

rt.members.append(Monkey(troop= rt, 
  items= [85, 91, 90], 
  op= lambda x : x + 4, 
  testDiv= 11, 
  trueMonkey= 3, 
  falseMonkey= 1)
)

rt.members.append(Monkey(troop= rt, 
  items= [67, 96, 69, 55, 70, 83, 62], 
  op= lambda x : x * 2, 
  testDiv= 3, 
  trueMonkey= 4, 
  falseMonkey= 1)
)

rt.members.append(Monkey(troop= rt, 
  items= [53, 86, 98, 70, 64], 
  op= lambda x : x + 6, 
  testDiv= 7, 
  trueMonkey= 7, 
  falseMonkey= 0)
)

rt.members.append(Monkey(troop= rt, 
  items= [88, 64], 
  op= lambda x : x * x, 
  testDiv= 17, 
  trueMonkey= 2, 
  falseMonkey= 5)
)

rt.runXrounds(10000)

--Round: 0 --
Monkey 0: [75, 81, 103, 83, 64, 82, 59, 92, 104, 76]
Monkey 1: [89, 95, 94, 80, 102, 74, 91, 134, 110, 140, 166, 124, 520]
Monkey 2: [23409]
Monkey 3: []
Monkey 4: [192, 138]
Monkey 5: [7744, 4096, 54756, 25281, 71289, 24336, 31329, 30276, 4900, 5929]
Monkey 6: []
Monkey 7: []
--Round: 100 --
Monkey 0: [4092690, 3088907, 6669149, 6732905, 1117295, 2013553, 143383, 7604683, 799357, 799357, 9348667, 8018407, 5906025]
Monkey 1: [2192356, 4026496, 3451216, 3204706, 6621196, 6621196, 2013540, 2013540, 799344, 3351894, 5618004, 7115184, 7115184, 5395502]
Monkey 2: [2013531, 143361]
Monkey 3: [6662469]
Monkey 4: [2192352, 6669132, 5986758, 3005472]
Monkey 5: [6863091, 2697751]
Monkey 6: []
Monkey 7: []
--Round: 200 --
Monkey 0: [1854300, 1854300, 6073427, 1872665, 8509295, 1958969, 3451229, 3451229, 7604683, 3351907, 5618017]
Monkey 1: [1872652, 8509282, 7482256, 8820316, 2013540, 5618004, 5618004, 3863972]
Monkey 2: [2524041]
Monkey 3: [1029622, 3088899]
Monkey 4: [6669132, 219